In [ ]:
import cv2
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import json

# Define paths (update these paths as needed)
input_images_folder = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\labeled_regions\inside"  # Path to input images
reel_mask_path = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian\filled_mask_below.png"  # Path to the reel mask
output_folder = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\labeled_regions\layer_detection"  # Path to save results
os.makedirs(output_folder, exist_ok=True)

# Load the reel mask
def load_mask(mask_path):
    """
    Load the mask for the reel.
    """
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        raise FileNotFoundError(f"Mask file not found at {mask_path}")
    return mask

# Preprocess image (apply mask, grayscale conversion)
def preprocess_image(image, mask):
    """
    Apply the reel mask and convert the image to grayscale.
    """
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    grayscale_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    return grayscale_image

# Detect cable edges
def detect_edges(grayscale_image):
    """
    Apply edge detection to identify cable boundaries.
    """
    edges = cv2.Canny(grayscale_image, 50, 150)  # Canny edge detection
    return edges

# Analyze edges to detect layers
def detect_layers(edges):
    """
    Detect layers on the reel using line detection.
    """
    # Use Hough Line Transform to detect lines
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=100)
    if lines is None:
        return []
    
    # Convert lines to readable format [(rho, theta)]
    detected_lines = []
    for rho, theta in lines[:, 0]:
        detected_lines.append((rho, theta))
    return detected_lines

# Count the layers based on detected lines
def count_layers(detected_lines):
    """
    Count the layers based on detected line patterns.
    """
    # Group lines into layers (simplified logic based on rho differences)
    layer_threshold = 20  # Minimum difference in rho to consider a new layer
    detected_rhos = sorted([line[0] for line in detected_lines])
    layers = [detected_rhos[0]]  # Initialize with the first line's rho

    for rho in detected_rhos[1:]:
        if abs(rho - layers[-1]) > layer_threshold:
            layers.append(rho)

    return len(layers)

# Visualize results
def visualize_results(image, edges, detected_lines, output_path):
    """
    Visualize the detected edges and lines overlaid on the original image.
    """
    # Draw edges
    edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    combined_image = cv2.addWeighted(image, 0.7, edges_colored, 0.3, 0)

    # Draw detected lines
    for rho, theta in detected_lines:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * a)
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * a)
        cv2.line(combined_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Save the visualization
    cv2.imwrite(output_path, combined_image)

# Main processing function
def process_images(input_folder, mask_path, output_folder):
    """
    Main function to process all images and detect cable layers.
    """
    mask = load_mask(mask_path)
    image_filenames = sorted([f for f in os.listdir(input_folder) if f.endswith(".jpg")])
    results = []

    for img_name in tqdm(image_filenames, desc="Processing Images"):
        img_path = os.path.join(input_folder, img_name)
        image = cv2.imread(img_path)
        if image is None:
            print(f"Image {img_name} could not be loaded. Skipping.")
            continue

        # Preprocess image
        grayscale = preprocess_image(image, mask)

        # Detect edges
        edges = detect_edges(grayscale)

        # Detect and count layers
        detected_lines = detect_layers(edges)
        num_layers = count_layers(detected_lines)

        # Save visualization
        visualization_path = os.path.join(output_folder, f"visualized_{img_name}")
        visualize_results(image, edges, detected_lines, visualization_path)

        # Save results
        results.append({"image": img_name, "layers_detected": num_layers})
    
    # Save results to JSON
    results_json_path = os.path.join(output_folder, "layer_detection_results.json")
    with open(results_json_path, "w") as json_file:
        json.dump(results, json_file, indent=4)

    print(f"Results saved to {results_json_path}")

# Run the script
process_images(input_images_folder, reel_mask_path, output_folder)
